In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from WickContractions.wick.contract import *
from WickContractions.laph.diagram import *

from utilities import *
from print_utilities import *

from IPython.display import display, Math

pprint = lambda o : display(Math(str(o)))

import os

## Operators

At the moment we are hardcoding in the operators.  We will want to read in a dictionary of spin matrices, as well as isospin flavor combinations.

In [3]:
# TODO get input from operator code for Isospin/momentum combinations.

In [4]:
# load definition of the building blocks of the operators.
from udud_elementals import *
import numpy as np

#gammas = [('buchoff',0)]
gammas=[]
gammas += np.array([[('A1g',i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('Eg{}'.format(row),i) for i in range(1)] for row in range(2)]).reshape(2,2).tolist()

print(gammas)

gammaName = lambda irrep,i : "\\Gamma^{{ ({},{}) }}".format(irrep, i)

aOps=[]
cOps=[]
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','u','u','u'],'x_0',gammaName(g[0],g[1]))]))
    cOps.append(Operator([baryonSource(1,['u','u','u','u'], 'x_1',gammaName(g[0],g[1]))]))

[['A1g', '0'], ['Eg0', '0'], ['Eg1', '0']]


## Wick Contractions

Now we run the wick contractions giving us essentially a list of tensor contractions to be performed.

In [5]:
diagrams=[]
for aOp in aOps:
    for cOp in cOps:
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)
print("Need {} diagrams".format(len(diagrams)))

Need 216 diagrams


In [6]:
laphDiagrams = [LDiagram(d) for d in diagrams]
simplify_laphDiagrams(laphDiagrams)
#for d in laphDiagrams:
#    pprint(d)

In [7]:
#allBaryonTensors = find_all_baryon_tensors(laphDiagrams)
# temporary TODO make smarter version of the rest of this notebook
allBaryonTensors={}
idx=0
for g in gammas:
    allBaryonTensors['B(x_1,t_i,\\Gamma^{{ ({},{}) }})'.format(g[0],g[1])]=idx
    allBaryonTensors['B^*(x_0,t_i,t_f,\\Gamma^{{ ({},{}) }})'.format(g[0],g[1])]=idx+1
    idx+=2

allBaryonTensorsBack={v: k for k,v in allBaryonTensors.items()}

In [8]:
allBaryonTensors

{'B(x_1,t_i,\\Gamma^{ (A1g,0) })': 0,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (A1g,0) })': 1,
 'B(x_1,t_i,\\Gamma^{ (Eg0,0) })': 2,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg0,0) })': 3,
 'B(x_1,t_i,\\Gamma^{ (Eg1,0) })': 4,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg1,0) })': 5}

In [9]:
allBaryonProps = {}
allBaryonSinks = {}
pIdx=0
sIdx=0
for k,v in allBaryonTensors.items():
    if k[0:3]=="B^*":
        allBaryonProps[k]=pIdx
        pIdx+=1
    else:
        allBaryonSinks[k]=sIdx
        sIdx+=1

In [10]:
allBaryonSinks

{'B(x_1,t_i,\\Gamma^{ (A1g,0) })': 0,
 'B(x_1,t_i,\\Gamma^{ (Eg0,0) })': 1,
 'B(x_1,t_i,\\Gamma^{ (Eg1,0) })': 2}

In [11]:
allBaryonProps

{'B^*(x_0,t_i,t_f,\\Gamma^{ (A1g,0) })': 0,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg0,0) })': 1,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg1,0) })': 2}

In [12]:
create_diagram_gpu_file(laphDiagrams, allBaryonTensors, allBaryonTensorsBack, allBaryonSinks, allBaryonProps)    
create_input_file()

In [13]:
for d in laphDiagrams[0:4]:
    pprint(d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Python code to combine diagrams

In [5]:
diagrams=contract(aOps[0], cOps[0]).diagrams

In [6]:
laphDiagrams = [LDiagram(d) for d in diagrams]
simplify_laphDiagrams(laphDiagrams)

In [13]:
s=""
for i,d in enumerate(laphDiagrams):
    s+="+{}*allDiags[i+{},t]".format(d.coef,i)
print(s)

+1.0*allDiags[i+0,t]+-1.0*allDiags[i+1,t]+1.0*allDiags[i+2,t]+-1.0*allDiags[i+3,t]+1.0*allDiags[i+4,t]+-1.0*allDiags[i+5,t]+1.0*allDiags[i+6,t]+-1.0*allDiags[i+7,t]+1.0*allDiags[i+8,t]+-1.0*allDiags[i+9,t]+1.0*allDiags[i+10,t]+-1.0*allDiags[i+11,t]+1.0*allDiags[i+12,t]+-1.0*allDiags[i+13,t]+1.0*allDiags[i+14,t]+-1.0*allDiags[i+15,t]+1.0*allDiags[i+16,t]+-1.0*allDiags[i+17,t]+1.0*allDiags[i+18,t]+-1.0*allDiags[i+19,t]+1.0*allDiags[i+20,t]+-1.0*allDiags[i+21,t]+1.0*allDiags[i+22,t]+-1.0*allDiags[i+23,t]
